# Creating DESDM DB tables

----

This is a short tutorial documenting the various techniques for creating tables in the DESDM Oracle DB using ```easyaccess```. This tutorial goes through the procedure of creating tables from the command line, the python API, and with explicit column typing via SQL commands.

In [1]:
%matplotlib inline

In [2]:
import subprocess
import numpy as np
import fitsio
from matplotlib.mlab import rec2csv
import easyaccess as ea

## Create data for upload

In [3]:
size = (100)
x = np.random.normal(size=size)
y = np.random.randint(0,size,size)
z = y.astype(str)
data = np.rec.fromarrays([x,y,z],names=['xcol','ycol','zcol'])

In [4]:
fitstable = "mytable_fits" 
fitsfile = fitstable+'.fits' 
fitsio.write(fitsfile,data,clobber=True)

csvtable = "mytable_csv" 
csvfile = csvtable+'.csv' 
rec2csv(data,csvfile)

## Uploading with Default Typing

Upload the tables from the command line using ```easyaccess```. The datatype conversion is done by the ```easyaccess``` default mapping.

### Command-line upload

In [5]:
command = "easyaccess -q -lt %s"%fitsfile
print subprocess.check_output(command,shell=True)


 Inserted 100 rows and 3 columns into table MYTABLE_FITS in 0.03 seconds

 Table MYTABLE_FITS loaded successfully.

 You might want to refresh the metadata (refresh_metadata_cache)
 so your new table appears during autocompletion

To make this table public run: 

   grant select on MYTABLE_FITS to DES_READER;  




In [6]:
command = "easyaccess -q -lt %s"%csvfile
print subprocess.check_output(command,shell=True)


 Inserted 100 rows and 3 columns into table MYTABLE_CSV in 0.17 seconds

 Table MYTABLE_CSV loaded successfully.

 You might want to refresh the metadata (refresh_metadata_cache)
 so your new table appears during autocompletion

To make this table public run: 

   grant select on MYTABLE_CSV to DES_READER;  




Now check that the upload succeeded. Notice that the ```zcol``` in the CSV file has been uploaded as a ```NUMBER``` because the formatting was not preserved when writing to the CSV file. For the FITS file, the typing of this column was properly preserved as ```VARCHAR2```. Note also that the column uploaded from the FITS file has whitespace characters padding the ```zcol``` entries (annoying).

In [7]:
command = 'easyaccess -q -c "describe_table %s;"'%(csvtable)
print subprocess.check_output(command,shell=True)

command = 'easyaccess -q -c "select * from %s where rownum < 10;"'%(csvtable)
print subprocess.check_output(command,shell=True)


Description of MYTABLE_CSV commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL  BINARY_DOUBLE         8,,     None
2        YCOL         NUMBER       22,,0     None
3        ZCOL         NUMBER       22,,0     None




9 rows in 0.03 seconds

       XCOL  YCOL  ZCOL
1 -0.795197    62    62
2  0.475525    65    65
3  1.632777    73    73
4  1.288126    40    40
5 -0.924750    64    64
6 -1.262833    52    52
7  0.187935    98    98
8 -0.071481    16    16
9  0.348446    61    61




In [8]:
command = 'easyaccess -q -c "describe_table %s;"'%(fitstable)
print subprocess.check_output(command,shell=True)

command = 'easyaccess -q -c "select * from %s where rownum < 10;"'%(fitstable)
print subprocess.check_output(command,shell=True)


Description of MYTABLE_FITS commented as: 'None'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL  BINARY_DOUBLE         8,,     None
2        YCOL         NUMBER        22,,     None
3        ZCOL       VARCHAR2        21,,     None




9 rows in 0.02 seconds

       XCOL  YCOL                   ZCOL
1 -0.795197    62  62                   
2  0.475525    65  65                   
3  1.632777    73  73                   
4  1.288126    40  40                   
5 -0.924750    64  64                   
6 -1.262833    52  52                   
7  0.187935    98  98                   
8 -0.071481    16  16                   
9  0.348446    61  61                   




### Python API upload

For more details on the ```easyaccess``` python API, see the notebook [here](https://github.com/mgckind/access_desdm/blob/master/easyaccess_api.ipynb). Below are the basics...

In [9]:
conn=ea.connect(section='dessci',quiet=True)

In [10]:
apitable = 'mytable_api'
conn.load_table(fitsfile,name=apitable)


 Inserted 100 rows and 3 columns into table MYTABLE_API in 0.05 seconds

 Table MYTABLE_API loaded successfully.

 You might want to refresh the metadata (refresh_metadata_cache)
 so your new table appears during autocompletion

To make this table public run: 

   grant select on MYTABLE_API to DES_READER;  



In [11]:
conn.mytables()


List of my tables

        TABLE_NAME  SIZE_GBYTES
1  FGOTTENMETADATA     0.002930
2      MYTABLE_API     0.000061
3     MYTABLE_FITS     0.000061
4      MYTABLE_CSV     0.000061



Clean up a bit (the ```PURGE``` command is necessary to empty Oracle's "recycling bin").

In [12]:
for table in [csvtable,fitstable,apitable]:
    query = "DROP TABLE %s PURGE"%(table)
    print query
    conn.cursor().execute(query)

DROP TABLE mytable_csv PURGE
DROP TABLE mytable_fits PURGE
DROP TABLE mytable_api PURGE


## Upload with Custom Typing

The idea is that we first create the table with a custom SQL command and then we append data to fill the table. The SQL creation command can be designed by hand, or it can be automated to some extent.

In [13]:
sqltable = "mytable_sql"
sql = """
--- Custom Table Creation
CREATE TABLE %s (
xcol NUMBER(6,4) NOT NULL,
ycol BINARY_DOUBLE,
zcol VARCHAR2(5)
);
"""%sqltable

sqlfile = sqltable+".sql"
out = open(sqlfile,'w')
out.write(sql)
out.close()

In [14]:
# Create the table from the python API
query = conn.loadsql(sqlfile)
conn.cursor().execute(query)
conn.onecmd("add_comment table %s 'This is a table created with explicit typing.'"%sqltable)

# This can also be done from the command line with:
# "easyaccess -q -loadsql %s"%sqlfile

Comment added to table: mytable_sql



In [15]:
conn.describe_table(sqltable)


Description of MYTABLE_SQL commented as: 'This is a table created with explicit typing.'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL         NUMBER      22,6,4     None
2        YCOL  BINARY_DOUBLE         8,,     None
3        ZCOL       VARCHAR2         5,,     None



Now we append the data file to the table that we just created.

In [16]:
conn.append_table(csvfile,sqltable)


 Inserted 100 rows and 3 columns into table MYTABLE_SQL in 0.05 seconds

 Table MYTABLE_SQL appended successfully.


In [17]:
conn.describe_table(sqltable)
conn.query_to_pandas("select * from %s where rownum < 10"%sqltable)


Description of MYTABLE_SQL commented as: 'This is a table created with explicit typing.'

  COLUMN_NAME      DATA_TYPE DATA_FORMAT COMMENTS
1        XCOL         NUMBER      22,6,4     None
2        YCOL  BINARY_DOUBLE         8,,     None
3        ZCOL       VARCHAR2         5,,     None



,XCOL,YCOL,ZCOL
0,-0.7952,62,62
1,0.4755,65,65
2,1.6328,73,73
3,1.2881,40,40
4,-0.9247,64,64
5,-1.2628,52,52
6,0.1879,98,98
7,-0.0715,16,16
8,0.3484,61,61


In [18]:
data = conn.query_to_pandas("select * from nsevilla.y1a1_gold_1_0_1 where rownum < 10").to_records()
# For some reason, query_to_pandas returns the index as the first column...
data.dtype.names[0]

'index'

In [19]:
def dtype2desdm(name,dtype):
    """
    This is a toy function for following some of the DESDM `conventions`
    for defining column types. The `conventions` come mostly from Y1A1_OBJECTS.
    
    This is NOT meant to be comprehensive. It is merely offered as an example
    of a possible typing procedure.
    """
    NAME = name.upper()
    if NAME == 'COADD_OBJECTS_ID':
        return "NUMBER(11,0)"
    elif NAME in ['RA','DEC','L','B']:
        return "NUMBER(9,6)"
    elif NAME.startswith(("SPREAD_","SPREADERR_","WAVG_SPREAD_")):
        return "NUMBER(6,5)"
    elif NAME.startswith(("MAG_","WAVG_MAG_","WAVGCALIB_MAG_","WAVG_MAGERR")):
        # WAVG_MAGERR is probably a mistake and should be with other MAGERR values
        return "NUMBER(6,4)"
    elif NAME.startswith(("CLASS_STAR","MAGERR_")):
        return "NUMBER(5,4)"
    elif NAME.startswith(('FLAGS_','OBSERVED_','MODEST_CLASS')):
        return "NUMBER(3,0)"
    elif NAME.startswith(('SLR_SHIFT','DESDM_ZP','DESDM_ZPERR')):
        return "NUMBER(6,4)"
    elif NAME == 'HPIX':
        return "NUMBER(10,0)"
    else:
        return ea.dtype2oracle(dtype)

In [20]:
autotable = "mytable_auto"
autofile = autotable+".sql"
out = open(autofile,'w')
out = "--- Automatically generated column typing\n"
out += "CREATE TABLE %s (\n"%autotable
for i,n in enumerate(data.dtype.names):
    # `index` is a protected name, so can't upload.
    if n == 'index': continue
    out+="  %s %s,\n"%(n,dtype2desdm(n,data.dtype.fields[n][0]))
    if i > 9: break
out = out.strip().strip(',')
out+='\n);\n'
print out
outfile = open(autofile,'w')
outfile.write(out)
outfile.close()

--- Automatically generated column typing
CREATE TABLE mytable_auto (
  COADD_OBJECTS_ID NUMBER(11,0),
  RA NUMBER(9,6),
  DEC NUMBER(9,6),
  L NUMBER(9,6),
  B NUMBER(9,6),
  SPREAD_MODEL_G NUMBER(6,5),
  SPREAD_MODEL_R NUMBER(6,5),
  SPREAD_MODEL_I NUMBER(6,5),
  SPREAD_MODEL_Z NUMBER(6,5),
  SPREAD_MODEL_Y NUMBER(6,5)
);



In [21]:
query = conn.loadsql(autofile)
conn.cursor().execute(query)
# Careful, `describe_table` does not list empty tables...
conn.query_and_print('select table_name from user_tables')



3 rows in 0.94 seconds

        TABLE_NAME
1      MYTABLE_SQL
2  FGOTTENMETADATA
3     MYTABLE_AUTO



In [22]:
for table in [sqltable,autotable]:
    query = "DROP TABLE %s PURGE"%(table)
    print query
    conn.cursor().execute(query)
conn.query_and_print('select table_name from user_tables')

DROP TABLE mytable_sql PURGE
DROP TABLE mytable_auto PURGE


1 rows in 0.33 seconds

        TABLE_NAME
1  FGOTTENMETADATA

